In [ ]:
!pip install colab_ssh --upgrade

In [ ]:
! pip install awscli
! pip install awssam
! aws configure
! sls deploy

In [ ]:
```mermaid
sequenceDiagram
    api/urls->>+selializeze: testtt?
    Alice->>+John: Hello John, how are you?
    Alice->>+John: John, can you hear me?
    Alice->>+selializeze: ssssss?
    John-->>-Alice: Hi Alice, I can hear you!
    John-->>-Alice: I feel great! https://jp
    John->>+test: aaaaaaa
    test->>-aaa: aaaa
```

```mermaid
sequenceDiagram
    api/urls->>+selializeze: testtt?
    Alice->>+John: Hello John, how are you?
    Alice->>+John: John, can you hear me?
    Alice->>+selializeze: ssssss?
    John-->>-Alice: Hi Alice, I can hear you!
    John-->>-Alice: I feel great! https://jp
    John->>+test: aaaaaaa
    test->>-aaa: aaaa
```

```mermaid
sequenceDiagram
    api/urls->>+selializeze: testtt?
    Alice->>+John: Hello John, how are you?
    Alice->>+John: John, can you hear me?
    Alice->>+selializeze: ssssss?
    John-->>-Alice: Hi Alice, I can hear you!
    John-->>-Alice: I feel great! https://jp
    John->>+test: aaaaaaa
    test->>-aaa: aaaa
```

! settings

In [ ]:
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="PASSWORD")

---
title: Brefを利用したサーバレスLaravelアプリケーションの構築
tags: PHP bref Laravel ServerlessFramework serverless
author: mgmgmogumi
slide: false
---
[Bref](https://bref.sh/)を理解するためにプロジェクト作り直して一から勉強中の忘備録。
まずはBrefの存在意義について。

### なぜBrefを使うのか

[AWS Lambda](https://docs.aws.amazon.com/lambda/)がサポートする言語は現時点(2021/8/31)で、**Java、Go、PowerShell、Node.js、C#、Python、Ruby**のみ。
これらの言語を使用する場合はLambda上でコードを書いて動かすことができるが、**PHPはサポートされていない。**

Lambdaは、その他の言語を使用できるようにするためランタイムAPIという機能を提供している。
PHPを使用するには、カスタムランタイム機能を使って自分でランタイム環境をビルドして、bootstrapというカスタムランタイムのエンドポイントとなるファイルを用意する必要がある。
これがまぁまぁ面倒くさい。

brefを使うと、その部分を[Serverless Framework](https://www.serverless.com/)を利用してサクッと自動構築してくれて、サーバーレスPHPアプリケーションを簡単に作成できる。

![3_bref_flow_ja.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/140953/c68d1144-7872-f91d-f4f9-a492e775020d.png)

https://aws.amazon.com/jp/blogs/news/the-serverless-lamp-stack-part-3-replacing-the-web-server/

以下は[Laravel](http://laravel.jp/)でのBref導入時の手順。

### 1. Serverless Frameworkのインストール

```
$ npm install -g serverless
$ serverless config credentials --provider aws --key <key> --secret <secret>
```

Serverless Frameworkとは、AWSならCloudFromationといったクラウドの構築管理サービスといい感じに連携してくれるOSS。(説明雑

### 2. Laravelプロジェクトを作成

```
$ composer create-project laravel/laravel sample-serverless-laravel
$ cd sample-serverless-laravel
```

今回は新しく作成しているけど既存のプロジェクトでもOK。

### 3. Bref&Laravel-Brefパッケージをインストール

```
$ composer require bref/bref
$ composer require bref/bref bref/laravel-bridge --update-with-dependencies
```

Laravel-Brefパッケージは、Laravelでのログドライバやセッションドライバの設定をサーバレス用にいい感じに設定してくれる。

### 4. serverless.yml構成ファイルを作成

```
$ php artisan vendor:publish --tag=serverless-config
```

```
Copied File [/vendor/bref/laravel-bridge/config/serverless.yml] To [/serverless.yml]
Publishing complete.
```

以下のようなファイルがプロジェクトディレクトリに作成される。

```serverless.yml
service: laravel

provider:
    name: aws
    # The AWS region in which to deploy (us-east-1 is the default)
    region: us-east-1
    # The stage of the application, e.g. dev, production, staging… ('dev' is the default)
    stage: dev
    runtime: provided.al2

package:
    # Directories to exclude from deployment
    exclude:
        - node_modules/**
        - public/storage
        - resources/assets/**
        - storage/**
        - tests/**

functions:
    # This function runs the Laravel website/API
    web:
        handler: public/index.php
        timeout: 28 # in seconds (API Gateway has a timeout of 29 seconds)
        layers:
            - ${bref:layer.php-74-fpm}
        events:
            -   httpApi: '*'
    # This function lets us run artisan commands in Lambda
    artisan:
        handler: artisan
        timeout: 120 # in seconds
        layers:
            - ${bref:layer.php-74} # PHP
            - ${bref:layer.console} # The "console" layer

plugins:
    # We need to include the Bref plugin
    - ./vendor/bref/bref

```

必要なら`region: us-east-1`の部分を`region: ap-northeast-1`(東京リージョン)に変える。


### 5. キャッシュクリア

プロジェクト作成直後は必要ないけど、二度目以降はキャッシュが残るのでデプロイ前にキャッシュクリアが必要。

```
$ php artisan config:clear
```

```
Configuration cache cleared!
```

### 6. デプロイ

```
$ serverless deploy
```

```
Service Information
service: laravel
stage: dev
region: us-east-1
stack: laravel-dev
resources: 14
api keys:
  None
endpoints:
  ANY - https://xxxxxxxxxx.execute-api.us-east-1.amazonaws.com
functions:
  web: laravel-dev-web
  artisan: laravel-dev-artisan
layers:
  None
```

### デプロイ確認

一応AWSコンソール上でもデプロイ確認。
AWS CloudFormationにアクセス。
イベントタブやリソースタブで各リソースが`CREATE_COMPLETE`または`UPDATE_COMPLETE`になっていればOK。

![スクリーンショット 2021-08-04 18.30.54.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/140953/7e77a4df-ccf3-84f1-699b-900f158715af.png)

### artisanコマンド確認

`vendor/bin/bref cli <function-name> -- <command>`

使用例↓

```
$ vendor/bin/bref cli laravel-dev-artisan -- inspire
```

誰かの名言が出てきたらOK。無事にbref上でphp artisanのコマンドが使えている。

記事に間違いあったらご指摘ください。
次回はRDSを構成して繋げる部分を書いていきます。

### 参考

https://bref.sh/docs/installation.html

https://bref.sh/docs/frameworks/laravel.html




